In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import resnet50, xception, inception_v3  
from keras import Input, Model, layers

In [ ]:
import pathlib

BATCH_SIZE = 32
IMG_SIZE = (224, 224)

data_dir = pathlib.Path('drive/My Drive/TIES4911/wonders_world/Wonders of World/Wonders of World')
(train_ds, val_ds) = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    shuffle=True,
    validation_split=0.2,
    subset='both',
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    seed=123,
    label_mode='categorical'
)

class_names = train_ds.class_names
num_classes = len(class_names)

Found 3846 files belonging to 12 classes.
Using 3077 files for training.
Using 769 files for validation.


In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)

In [ ]:
for x_batch, y_batch in train_ds.take(1):
  print(x_batch.shape)
  break

(32, 180, 180, 3)


In [ ]:
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.RandomFlip('horizontal'),
  tf.keras.layers.RandomRotation(0.2),
])

In [ ]:
inputs = Input(shape=(224, 224, 3))

base_model = resnet50.ResNet50(include_top=False)
base_model.trainable = False

preprocess_input = resnet50.preprocess_input
global_average_layer = layers.GlobalAveragePooling2D()
prediction_layer = layers.Dense(12)

x = data_augmentation(inputs)
x = preprocess_input(x)
x = base_model(x, training=False)
x = global_average_layer(x)
x = layers.Dropout(0.5)(x)
outputs = prediction_layer(x)

model = Model(inputs, outputs)
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

checkpoint = tf.train.Checkpoint(model=model)
model.summary()

loss0, accuracy0 = model.evaluate(val_ds)

Model: "model_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_18 (InputLayer)       [(None, 224, 224, 3)]     0         
                                                                 
 sequential_1 (Sequential)   (None, 224, 224, 3)       0         
                                                                 
 tf.__operators__.getitem_8   (None, 224, 224, 3)      0         
 (SlicingOpLambda)                                               
                                                                 
 tf.nn.bias_add_8 (TFOpLambd  (None, 224, 224, 3)      0         
 a)                                                              
                                                                 
 resnet50 (Functional)       (None, None, None, 2048)  23587712  
                                                                 
 global_average_pooling2d_1   (None, 2048)             0   

In [ ]:
print(len(model.trainable_variables))

In [ ]:
checkpoint_path = 'drive/My Drive/TIES4911/task4/task4-1/resnet50-{epoch:04d}'

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=True,
    save_freq=2*97,
    verbose=1)

history = model.fit(train_ds,
                    epochs=10,
                    validation_data=val_ds,
                    callbacks=[model_checkpoint_callback])

Epoch 1/10


97/97 [==============================] - 854s 9s/step - loss: 8.6333 - accuracy: 0.1118 - val_loss: 10.6517 - val_accuracy: 0.1925
Epoch 2/10
96/97 [============================>.] - ETA: 6s - loss: 10.0396 - accuracy: 0.1396 
Epoch 2: saving model to drive/My Drive/TIES4911/task4/task4-1/resnet50-0002
97/97 [==============================] - 784s 8s/step - loss: 10.0391 - accuracy: 0.1394 - val_loss: 9.6221 - val_accuracy: 0.1482
Epoch 3/10
97/97 [==============================] - 825s 9s/step - loss: 10.4756 - accuracy: 0.1241 - val_loss: 11.7640 - val_accuracy: 0.1717
Epoch 4/10
96/97 [============================>.] - ETA: 6s - loss: 10.2530 - accuracy: 0.1315 
Epoch 4: saving model to drive/My Drive/TIES4911/task4/task4-1/resnet50-0004
97/97 [==============================] - 816s 8s/step - loss: 10.2626 - accuracy: 0.1313 - val_loss: 9.5494 - val_accuracy: 0.1717
Epoch 5/10
97/97 [==============================] - 813s 8s/step - loss: 10.5570 - accuracy: 0.1440 - val_loss: 11.676

In [ ]:
loss1, accuracy1 = model.evaluate(val_ds)

25/25 [==============================] - 165s 6s/step - loss: 9.8205 - accuracy: 0.1547
